# Getting started - level 1

Let's write `Hello World` program using quantum serverless. 

We will start with writing code for our program and saving it to [./source_files/gs_level_1.py](./source_files/gs_level_1.py) file. Our program will be a Qiskit hello world example, which prepares a Bell state and then returns the measured probability distribution

```python
# source_files/gs_level_1.py

from qiskit import QuantumCircuit
from qiskit.primitives import Sampler

circuit = QuantumCircuit(2)
circuit.h(0)
circuit.cx(0, 1)
circuit.measure_all()
circuit.draw()

sampler = Sampler()

quasi_dists = sampler.run(circuit).result().quasi_dists

print(f"Quasi distribution: {quasi_dists[0]}")
```

Next we need to run this program. For that we need to import necessary classes and configure them. 
One of those classes is `QuantumServerless`, which is a client class to interact with compute resources.
It will help us run programs, monitor progress and fetch results.

`QuantumServerless` accepts `Provider` as a constructor argument. Provider stores configuration where our compute resources are and how to connect to them. For this example we will be using provider which is connected to local docker-compose setup. For more information on docker-compose check out [docker docs](https://docs.docker.com/compose/), but for now you can think of it as your local environment manager. So, in this example programs will be running locally on your machine. If you want to run it elsewhere, you need to provide corresponding host and authentication details.

In [15]:
from quantum_serverless import QuantumServerless, GatewayProvider

In [16]:
provider = GatewayProvider(
    username="user", # this username has already been defined in local docker setup and does not need to be changed
    password="password123", # this password has already been defined in local docker setup and does not need to be changed
    host="http://gateway:8000", # address of provider
)

serverless = QuantumServerless(provider)
serverless

<QuantumServerless | providers [gateway-provider]>

Now we need to run our program file, by creating an instance of `Program` and calling `run_program` method of our `QuantumServerless` client.

`Program` accepts couple of required parameters:
- title - name of the program
- entrypoint - name of python file you want to execute
- working_dir - folder where  your script is located. This is optional parameter and will be current folder by default. 

In [17]:
from quantum_serverless import Program

program = Program(
    title="Getting started program level 1", # you can choose any name you like. It is used to differentiate if you have a lot of programs in array.
    entrypoint="gs_level_1.py", # entrypoint is file that will start your calculation
    working_dir="./source_files/" # where you files are located. By default it is current directory.
)

job = serverless.run_program(program)
job

<Job | aae3533f-ed0d-4e93-9247-4705d476ee97>

As result of `run` call we get `Job` which has `status` method to check status of program execution, `logs` to get logs of execution.

In [19]:
job.status()

'SUCCEEDED'

In [14]:
job.logs()

'Quasi distribution: {0: 0.4999999999999999, 3: 0.4999999999999999}\n'

Also this object has `job_id` property that can be used if you want to access job results later.
To do so we need to call `get_job_by_id` method of `QuantumServerless` client.

In [20]:
job.job_id

'aae3533f-ed0d-4e93-9247-4705d476ee97'

Users can fetch previously ran jobs from configured providers.

In [21]:
serverless.get_job_by_id(job.job_id)

<Job | aae3533f-ed0d-4e93-9247-4705d476ee97>